<a href="https://colab.research.google.com/github/kavyakalivarapu-lab/kavya.github.io/blob/main/CB_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract

In [2]:
!pip install langdetect

In [3]:
!pip install googletrans

In [4]:
!pip install indic-transliteration

In [6]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 12.0 MB/s eta 0:00:00


In [8]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 13.3 MB/s eta 0:00:00


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
import numpy as np
from torchvision import models, transforms
from transformers import BertTokenizer, BertModel, VisionEncoderDecoderModel, TrOCRProcessor
from PIL import Image
import cv2
import pytesseract
import re
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models.fasttext import FastText
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import editdistance
from urllib.parse import urlparse
from langdetect import detect
from googletrans import Translator
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
import contractions
import emoji
from textblob import TextBlob
from craft_text_detector import Craft  # CRAFT for text detection

In [10]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [11]:
from google.colab import drive
drive.mount('/content/drive')
# Path to dataset
DATASET_PATH = "/content/drive/MyDrive/DL/project"
IMAGE_FOLDER = os.path.join(DATASET_PATH, "bully_data")

# Load CSV with labels
df = pd.read_csv(os.path.join(DATASET_PATH, "Cyberbully_corrected_emotion_sentiment.xlsx.csv"))

# Add full image paths
df["image_path"] = df["Img-Name"].astype(str).apply(lambda x: os.path.join(IMAGE_FOLDER, x))

# Preview dataset
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Img-Name                                           Img-Text Img-Text-Label  \
0    0.jpg  Shivam @shivamishraa Girls be named naina and ...          Bully   
1    1.jpg  Aaloo ke paranthe is the best breakfast Omelet...       Nonbully   
2    2.jpg     For Boyfriend For Bestfriend DESI ADUKT TROLLS          Bully   
3    3.jpg  You find a new YouTuber He's funny All of his ...       Nonbully   
4    4.jpg  not_shubham14 @mentally_dank Kids at Marine Dr...          Bully   

  Img-Label Text-Label Sentiment   Emotion Sarcasm      Harmful-Score  \
0  Nonbully      Bully  Negative   Disgust     Yes  Partially-Harmful   
1  Nonbully   Nonbully   Neutral     Other      No           Harmless   
2     Bully   Nonbully  Negative  Ridicule      No  Partially-Harmful   
3  Nonbully   Nonbully   Neutral   Sadness      No           Harmless   
4  Nonbully   Nonbully  N

In [17]:
# Debugging: Check if dataset and images exist
print("Is Google Drive mounted?", os.path.exists("/content/drive/MyDrive"))
print("Does dataset folder exist?", os.path.exists(IMAGE_FOLDER))
print("Sample image path:", df["image_path"].iloc[0])
print("Exists?", os.path.exists(df["image_path"].iloc[0]))

Is Google Drive mounted? True
Does dataset folder exist? True
Sample image path: /content/drive/MyDrive/DL/project/bully_data/0.jpg
Exists? True


In [18]:
def clean_data(dataframe):
    print("Before Cleaning:", len(dataframe))

    # Ensure paths are valid
    dataframe = dataframe[dataframe['image_path'].notna()]

    # Check if at least some images exist
    existing_paths = dataframe['image_path'].apply(lambda x: os.path.exists(str(x)))

    if existing_paths.sum() == 0:
        print("⚠️ No valid image paths found! Skipping path filtering.")
        return dataframe  # Return unfiltered dataframe

    dataframe = dataframe[existing_paths]

    print("After Cleaning:", len(dataframe))
    return dataframe


In [19]:
df = clean_data(df)

Before Cleaning: 5865
After Cleaning: 5855


In [ ]:
class TextPreprocessor:
    def __init__(self):
        self.translator = Translator()
        self.slang_dict = {"u": "you", "lol": "laughing out loud", "brb": "be right back"}
        self.stop_words = set(stopwords.words("english"))
        self.nlp = spacy.load("en_core_web_sm")
        self.common_hindi_words = {"mera", "naam", "kya", "hai", "tum", "kaise", "ho", "badiya", "bhai"}
        self.suspicious_domains = {"badsite.com", "malwarepage.net", "phishingsite.org"}  # Example blacklist

    def detect_language(self, text):
        try:
            return detect(text)
        except:
            return "unknown"

    def transliterate_to_hindi(self, text):
        return transliterate(text, sanscript.ITRANS, sanscript.DEVANAGARI)

    def is_romanized_hindi(self, text):
        words = text.lower().split()
        return any(word in self.common_hindi_words for word in words)

    def translate_text(self, text):
        lang = self.detect_language(text)
        if lang == "hi" or self.is_romanized_hindi(text):
            hindi_text = self.transliterate_to_hindi(text)
            return self.translator.translate(hindi_text, src="hi", dest="en").text
        return text

    def expand_contractions(self, text):
        return contractions.fix(text)

    def correct_spelling(self, text):
        return str(TextBlob(text).correct())

    def clean_whitespace(self, text):
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'([!?.,]){2,}', r'\1', text)
        return text.strip()

    def convert_emojis(self, text):
        return emoji.demojize(text)

    def replace_slang(self, text):
        words = text.split()
        return " ".join([self.slang_dict[word] if word in self.slang_dict else word for word in words])

    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r"http\S+|www\S+", "", text)
        text = re.sub(r"@[A-Za-z0-9_]+", "", text)
        text = re.sub(r"#[A-Za-z0-9_]+", "", text)
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
        return text.strip()

    def remove_stopwords(self, text):
        words = word_tokenize(text)
        return " ".join([word for word in words if word not in self.stop_words])

    def lemmatize_text(self, text):
        doc = self.nlp(text)
        return " ".join([token.lemma_ for token in doc])

    def extract_domain(self, text):
        if not isinstance(text, str):
            return []
        urls = re.findall(r'www\.[a-zA-Z0-9.-]+', text)
        domains = [urlparse("http://" + url).netloc for url in urls]
        return domains

    def is_suspicious_domain(self, domain_list):
        return any(domain in self.suspicious_domains for domain in domain_list)

    def preprocess(self, text):
        text = self.expand_contractions(text)
        text = self.correct_spelling(text)
        text = self.clean_whitespace(text)
        text = self.convert_emojis(text)
        text = self.replace_slang(text)
        text = self.translate_text(text)
        text = self.clean_text(text)
        text = self.remove_stopwords(text)
        text = self.lemmatize_text(text)
        return text

In [ ]:
# Initialize Preprocessor
text_preprocessor = TextPreprocessor()


In [ ]:
df['Img-Text'] = df['Img-Text'].fillna("")
df["Img-Text"] = df["Img-Text"].apply(text_preprocessor.preprocess)
df["domains"] = df["Img-Text"].apply(text_preprocessor.extract_domain)
df["is_suspicious"] = df["domains"].apply(text_preprocessor.is_suspicious_domain)

In [ ]:
print("✅ Text Preprocessing & URL Analysis Integrated in Class!")

In [ ]:
class ImagePreprocessor:
    def __init__(self):
        pass

    def convert_to_grayscale(self, image):
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    def apply_threshold(self, image):
        _, thresh = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return thresh

    def remove_noise(self, image):
        return cv2.fastNlMeansDenoising(image, h=30)

    def apply_morphology(self, image):
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        return cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

    def apply_canny_edge(self, image):
        return cv2.Canny(image, 100, 200)

    def preprocess(self, image_path):
        image = cv2.imread(image_path)
        image = self.convert_to_grayscale(image)
        image = self.apply_threshold(image)
        image = self.remove_noise(image)
        image = self.apply_morphology(image)
        image = self.apply_canny_edge(image)
        return Image.fromarray(image)

In [ ]:
# Initialize Image Preprocessor
image_preprocessor = ImagePreprocessor()

In [ ]:
print("✅ Image Preprocessing Integrated!")

In [ ]:
class TextExtractor:
    def __init__(self):
        self.craft = Craft(output_dir='craft_output', crop_type="poly", cuda=False)  # Using CRAFT for text detection
        self.processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
        self.model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

    def detect_text_regions(self, image_path):
        return self.craft.detect_text(image_path)

    def extract_text_tesseract(self, image):
        return pytesseract.image_to_string(image, lang='eng')

    def extract_text_trocr(self, image):
        image = self.processor(image, return_tensors="pt").pixel_values
        generated_ids = self.model.generate(image)
        return self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    def extract_text(self, image_path):
        detected_regions = self.detect_text_regions(image_path)
        if detected_regions:
            image = Image.open(image_path)
            trocr_text = self.extract_text_trocr(image)
            tesseract_text = self.extract_text_tesseract(image)
            return trocr_text + " " + tesseract_text
        else:
            return ""

In [ ]:
# Initialize Text Extractor
text_extractor = TextExtractor()

In [ ]:
print("✅ Text Extraction Integrated!")

In [26]:
# Convert Domains into Machine Learning Features
vectorizer = CountVectorizer()
domain_features = vectorizer.fit_transform(df['domains'].astype(str))

In [28]:
# Remove ScoopWhoop URLs from text
df["Img-Text"] = df["Img-Text"].apply(lambda x: re.sub(r'\bSCOOPWHOOP\.com\b', '', x, flags=re.IGNORECASE))

In [40]:
# Train-Test Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [41]:
# Text Feature Extraction
vectorizer = TfidfVectorizer()
fasttext_model = FastText(sentences=df['cleaned_text'].apply(lambda x: x.split()), vector_size=100, window=5, min_count=1, workers=4)
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [42]:
def analyze_text(text):
    tfidf_features = vectorizer.fit_transform([text]).toarray()
    fasttext_vector = np.mean([fasttext_model.wv[word] for word in text.split() if word in fasttext_model.wv] or [np.zeros(100)], axis=0)
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        bert_features = bert_model(**inputs).pooler_output.squeeze(0).numpy()
    return tfidf_features, fasttext_vector, bert_features